In [1]:
import optlang
import optlang_enumerator.cobra_cnapy
import optlang_enumerator.mcs_computation as mcs_computation
import numpy
from pathlib import Path
# results_cache_dir = None # do not cache preprocessing results
results_cache_dir = Path(".") # cache preprocessing results (in the current directory)

In [2]:
ecc2 = optlang_enumerator.cobra_cnapy.CNApyModel.read_sbml_model("ECC2comp.sbml")
# allow all reactions that are not boundary reactions as cuts (same as exclude_boundary_reactions_as_cuts option of compute_mcs)
cuts = numpy.array([not r.boundary for r in ecc2.reactions])
reac_id = ecc2.reactions.list_attr('id') # list of reaction IDs in the model
# define target (multiple targets are possible; each target can have multiple linear inequality constraints)
ecc2_mue_target = [[("Growth", ">=", 0.01)]] # one target with one constraint, a.k.a. syntehtic lethals
# this constraint alone would not be sufficient, but there are uptake limits defined in the reaction bounds
# of the model that are integerated below, first, however, parse the constraint(s) and convert to matrix format
ecc2_mue_target = [mcs_computation.relations2leq_matrix(
                   mcs_computation.parse_relations(t, reac_id_symbols=mcs_computation.get_reac_id_symbols(reac_id)), reac_id)
                   for t in ecc2_mue_target]
# now integrate the non-default reaction bounds from the network
mcs_computation.integrate_model_bounds(ecc2, ecc2_mue_target)
# this is just to show what the first (and only) target now looks like:
for c in mcs_computation.get_leq_constraints(ecc2, ecc2_mue_target)[0]:
    print(c, c.problem)

Restricted license - for non-production use only - expires 2025-11-24
eed2d55c-354f-11ef-8715-004e01c408d4_0_0: -1.0*Growth + 1.0*Growth_reverse_699ae <= -0.01 None
eed2d55c-354f-11ef-8715-004e01c408d4_0_1: 1.0*AcUp - 1.0*AcUp_reverse_36302 <= 10.0 None
eed2d55c-354f-11ef-8715-004e01c408d4_0_2: 1.0*CO2Up - 1.0*CO2Up_reverse_9ac6c <= 10.0 None
eed2d55c-354f-11ef-8715-004e01c408d4_0_3: 1.0*GlcUp - 1.0*GlcUp_reverse_f2a50 <= 10.0 None
eed2d55c-354f-11ef-8715-004e01c408d4_0_4: 1.0*GlycUp - 1.0*GlycUp_reverse_d1488 <= 10.0 None
eed2d55c-354f-11ef-8715-004e01c408d4_0_5: 1.0*SuccUp - 1.0*SuccUp_reverse_79869 <= 10.0 None


In [3]:
# calculate MCS up to size 3 with "any MCS" enumeration method
ecc2_mcs,_ = mcs_computation.compute_mcs(ecc2, ecc2_mue_target, cuts=cuts, enum_method=3, max_mcs_size=3, network_compression=True,
                                         include_model_bounds=False, results_cache_dir=results_cache_dir)
print(len(ecc2_mcs), "MCS found.")
# show MCS as n-tuples of reaction IDs
ecc2_mcs_rxns= [tuple(reac_id[r] for r in mcs) for mcs in ecc2_mcs]
print(ecc2_mcs_rxns)
# check that all MCS disable the first (and only) target
print(all(mcs_computation.check_mcs(ecc2, ecc2_mue_target[0], ecc2_mcs, optlang.interface.INFEASIBLE)))

Loaded compressed model from CNA_ECC2comp_subsets_compressed_75890a457ad63e16157884b633ad1014
Using big M.
knock_in_idx []
Objective function is empty; set objective to self.minimize_sum_over_z
Found solution with objective value 2.0
CS ['{TALA|TKT1}' 'TKT2'] -> MCS ['{TALA|TKT1}' 'TKT2']
Found solution with objective value 3.0
CS ['{EX_glc_DASH_D_ex|GlcUp}' '{EX_glyc_ex|GlycUp}' '{EX_h_ex|h_pEx}'] -> MCS ['{EX_glc_DASH_D_ex|GlcUp}' '{EX_glyc_ex|GlycUp}' '{EX_h_ex|h_pEx}']
Found solution with objective value 3.0
CS ['{EX_o2_ex|CYTBO3_4pp|O2Up}' 'ATPS4rpp' '{GAPD|PGK}'] -> MCS ['{EX_o2_ex|CYTBO3_4pp|O2Up}' '{GAPD|PGK}']
Found solution with objective value 1.0
MCS ['ICDHyr']
Found solution with objective value 2.0
CS ['AcEx' '{ACKr|PTAr}'] -> MCS ['AcEx' '{ACKr|PTAr}']
Found solution with objective value 1.0
MCS ['ADK1']
Found solution with objective value 1.0
MCS ['{ACONTa|ACONTb|CS}']
Found solution with objective value 3.0
CS ['{EX_glyc_ex|GlycUp}' 'RPI' '{TALA|TKT1}'] -> MCS ['RPI']


In [4]:
# %% same calculation without network compression
ecc2_mcsF,_ = mcs_computation.compute_mcs(ecc2, ecc2_mue_target, cuts=cuts, enum_method=3, max_mcs_size=3, network_compression=False,
                                          include_model_bounds=False, results_cache_dir=results_cache_dir)
print(set(ecc2_mcs) == set(ecc2_mcsF)) # check that results agree

FVA to find blocked reactions...
Loaded FVA result from CNA_ECC2comp_FVA_d799bfc002fe911c23c1aed3a650a37b
Found 5 blocked reactions:
 ['EX_adp_c', 'EX_mqn8_c', 'EX_nad_c', 'EX_nadp_c', 'EX_q8_c']
Using big M.
knock_in_idx []
Objective function is empty; set objective to self.minimize_sum_over_z
Found solution with objective value 3.0
CS ['TKT1' 'TKT2' 'TPI'] -> MCS ['TKT1' 'TKT2']
Found solution with objective value 1.0
MCS ['ICDHyr']
Found solution with objective value 1.0
MCS ['CS']
Found solution with objective value 3.0
CS ['CYTBO3_4pp' 'FBA' 'PGM'] -> MCS ['CYTBO3_4pp' 'PGM']
Found solution with objective value 3.0
CS ['FBA' 'GAPD' 'PGM'] -> MCS ['GAPD' 'PGM']
Found solution with objective value 3.0
CS ['ATPS4rpp' 'CO2Ex' 'h_pEx'] -> MCS ['CO2Ex' 'h_pEx']
Found solution with objective value 3.0
CS ['CYTBO3_4pp' 'FBA' 'GAPD'] -> MCS ['CYTBO3_4pp' 'GAPD']
Found solution with objective value 3.0
CS ['GlcUp' 'PGM' 'TPI'] -> MCS ['GlcUp' 'PGM' 'TPI']
Found solution with objective value

In [5]:
# %% define a desired behaviour for cMCS calculation
ecc2_ethanol_desired = [[("EthEx", ">=", 1)]]
ecc2_ethanol_desired = [mcs_computation.relations2leq_matrix(
                   mcs_computation.parse_relations(t, reac_id_symbols=mcs_computation.get_reac_id_symbols(reac_id)), reac_id)
                   for t in ecc2_ethanol_desired]
mcs_computation.integrate_model_bounds(ecc2, ecc2_ethanol_desired)
ecc2_ethanol_desired_constraints= mcs_computation.get_leq_constraints(ecc2, ecc2_ethanol_desired)
for c in ecc2_ethanol_desired_constraints[0]: # print constraints that make up the desired region
    print(c)


0b06a99c-3550-11ef-8715-004e01c408d4_0_0: -1.0*EthEx + 1.0*EthEx_reverse_aaa12 <= -1.0
0b06a99c-3550-11ef-8715-004e01c408d4_0_1: 1.0*AcUp - 1.0*AcUp_reverse_36302 <= 10.0
0b06a99c-3550-11ef-8715-004e01c408d4_0_2: 1.0*CO2Up - 1.0*CO2Up_reverse_9ac6c <= 10.0
0b06a99c-3550-11ef-8715-004e01c408d4_0_3: 1.0*GlcUp - 1.0*GlcUp_reverse_f2a50 <= 10.0
0b06a99c-3550-11ef-8715-004e01c408d4_0_4: 1.0*GlycUp - 1.0*GlycUp_reverse_d1488 <= 10.0
0b06a99c-3550-11ef-8715-004e01c408d4_0_5: 1.0*SuccUp - 1.0*SuccUp_reverse_79869 <= 10.0


In [6]:
# %% calculate cMCS
ecc2_cmcs,_ = mcs_computation.compute_mcs(ecc2, ecc2_mue_target, desired=ecc2_ethanol_desired, cuts=cuts, enum_method=3, max_mcs_size=3, network_compression=True,
                                          include_model_bounds=False, results_cache_dir=results_cache_dir)

# %% check cMCS
print(len(ecc2_cmcs))
# all cMCS disable the target
print(all(mcs_computation.check_mcs(ecc2, ecc2_mue_target[0], ecc2_cmcs, optlang.interface.INFEASIBLE)))
# all cMCS allow the desired behaviour
print(all(mcs_computation.check_mcs(ecc2, ecc2_ethanol_desired[0], ecc2_cmcs, optlang.interface.OPTIMAL)))
# cMCS are a subset of the MCS 
print(set(ecc2_cmcs).issubset(ecc2_mcs))
# cMCS are those MCS that preserve the desired behaviour
print(set(ecc2_cmcs) ==
    set(m for m,c in zip(ecc2_mcs, mcs_computation.check_mcs(ecc2, ecc2_ethanol_desired[0], ecc2_mcs, optlang.interface.OPTIMAL)) if c))


Loaded compressed model from CNA_ECC2comp_subsets_compressed_75890a457ad63e16157884b633ad1014
Running FVA for desired regions...
Loaded FVA result from CNA_ECC2comp_FVA_8cc4436bd2392239946de888ef2c5b95
1 essential reactions in desired region 0
Using big M.
knock_in_idx []
Objective function is empty; set objective to self.minimize_sum_over_z
Found solution with objective value 1.0
MCS ['{EX_Biomass|EX_4CRSOL_ex|EX_5DRIB_ex|EX_AMOB_ex|EX_ca2_ex|EX_cl_ex|EX_coa_c|EX_cobalt2_ex|EX_cu2_ex|EX_fe2_ex|EX_fe3_ex|EX_k_ex|EX_meoh_ex|EX_mg2_ex|EX_mn2_ex|EX_mobd_ex|EX_MTHTHF_ex|EX_nh4_ex|EX_ni2_ex|EX_pi_c|EX_pi_ex|EX_so4_ex|EX_zn2_ex|Growth}']
Found solution with objective value 2.0
CS ['{ACONTa|ACONTb|CS}' 'SUCOAS'] -> MCS ['{ACONTa|ACONTb|CS}']
Found solution with objective value 2.0
CS ['{ENO|PGM}' '{GAPD|PGK}'] -> MCS ['{ENO|PGM}' '{GAPD|PGK}']
Found solution with objective value 3.0
CS ['MDH' 'PPC' 'SUCOAS'] -> MCS ['MDH' 'PPC']
Found solution with objective value 2.0
CS ['RPE' 'RPI'] -> MCS 